In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
def per_game_stats(year, lst):
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'
    
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content,'html.parser')
    
    table = soup.find_all(class_="full_table")
    
    head = soup.find(class_="thead")
    
    column_names = [head.text for item in head][0].replace("\n",",").split(",")[2:-1] #Get rid of 'Rk' and empty first col
    
    players = []
    
    for i in range(len(table)):
        
        player_ = []
        
        for td in table[i].find_all("td"):
            player_.append(td.text)
        players.append(player_)
        
    df = pd.DataFrame(players, columns = column_names)
    df['Season'] = f'{year - 1}-{year}'
    lst.append(df)

In [3]:
year = 2021
all_year = []
while year >= 1950:
    per_game_stats(year, all_year)
    year -= 1
    
all_year = pd.concat(all_year)
all_year = all_year.reset_index()
del all_year['index']
all_year['Player'] = all_year['Player'].map(lambda x: x.rstrip('*'))

In [4]:
all_year.to_csv(f'{year+1}-2021_per_game_stats.csv', header=True)